<h1>Digit Recognizer Keras Implementation for Kaggle Competition</h1>

<h3>Dependecies</h3>

In [1]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import OneHotEncoder

from keras.utils.np_utils import to_categorical

/Library/Frameworks/Python.framework/Versions/3.6/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


<h3>Load the Datasets</h3>

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [3]:
y_train = train['label']


x_train = train.drop(labels = ["label"], axis = 1)


del train

<h3>Normalize the Data</h3>

In [4]:
x_train = x_train/255.0
test = test/255.0

<h3>Reshape the data from 1D Array to 3D Matrices</h3>

In [5]:
x_train = x_train.values.reshape(-1,28,28,1)
test = test.values.reshape(-1,28,28,1)

<h3>Convert categorical values to one hot arrays</h3>

In [6]:
y_train = y_train.values.reshape(-1,1)
y_train = to_categorical(y_train)
y_train[0]

array([0., 1., 0., 0., 0., 0., 0., 0., 0., 0.])

<h3>Split the dataset into train and validation</h3>

In [7]:
random_seed = 2

In [8]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=random_seed)

<h1>Keras Convolutional Neural Network Modelling</h1>

<h3>Dependencies</h3>

In [9]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPool2D
from keras.optimizers import RMSprop
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import ModelCheckpoint

<h3>Model Initialization</h3>

In [10]:
model = Sequential()

<h3>Convolution And Pooling Layers</h3>

In [11]:
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu', input_shape = (28,28,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2)))
model.add(Dropout(0.25))


model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', activation ='relu'))
model.add(MaxPool2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

<h3>Flatten</h3>

In [12]:
model.add(Flatten())

<h3>Hidden and Output Layer</h3>

In [13]:
model.add(Dense(512, kernel_initializer='uniform', activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

<h3>Model Optimizer</h3>

In [14]:
optimizer = RMSprop(lr=0.001, rho=0.9, epsilon=1e-08, decay=0.0)

<h3>Model Compilation</h3>

In [15]:
model.compile(optimizer = optimizer , loss = "categorical_crossentropy", metrics=["accuracy"])

<h3>LROnPlateau Initialization</h3>

In [16]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [17]:
epochs = 5
batch_size = 86

<h3>We use Keras ImageDataGenerator in order to Augment the dataset</h3>

In [18]:
datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        zoom_range = 0.1, # Randomly zoom image 
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=False,  # randomly flip images
        vertical_flip=False)  # randomly flip images

datagen.fit(x_train)

<h3>Model Checkpoint Keras Callback Function</h3>

In [19]:
filepath = 'weights_best.hdf5'
checkpoint = ModelCheckpoint(filepath=filepath,monitor='val_acc', verbose = 1, save_best_only=True)

<h3>List of Callbacks</h3>

In [20]:
callbacklist = [learning_rate_reduction, checkpoint]

<h3>Fitting the Model</h3>

In [21]:
model.fit(x_train,y_train, epochs=epochs, validation_data=(x_val,y_val), batch_size=batch_size, callbacks = [learning_rate_reduction, checkpoint])

Train on 37800 samples, validate on 4200 samples
Epoch 1/5
37800/37800 [==============================] - 334s 9ms/step - loss: 0.2210 - acc: 0.9283 - val_loss: 0.0657 - val_acc: 0.9800

Epoch 00001: val_acc improved from -inf to 0.98000, saving model to weights_best.hdf5
Epoch 2/5
37800/37800 [==============================] - 315s 8ms/step - loss: 0.0677 - acc: 0.9796 - val_loss: 0.0550 - val_acc: 0.9840

Epoch 00002: val_acc improved from 0.98000 to 0.98405, saving model to weights_best.hdf5
Epoch 3/5
37800/37800 [==============================] - 347s 9ms/step - loss: 0.0490 - acc: 0.9856 - val_loss: 0.0320 - val_acc: 0.9912

Epoch 00003: val_acc improved from 0.98405 to 0.99119, saving model to weights_best.hdf5
Epoch 4/5
37800/37800 [==============================] - 341s 9ms/step - loss: 0.0414 - acc: 0.9877 - val_loss: 0.0420 - val_acc: 0.9895

Epoch 00004: val_acc did not improve
Epoch 5/5
37800/37800 [==============================] - 363s 10ms/step - loss: 0.0354 - acc: 0.98

<h3>Make the Predictions based on trained Model</h3>

<h3>Dependencies</h3>

In [22]:
from keras.models import load_model

<h3>Load the best Model</h3>

In [23]:
best_model = load_model('weights_best.hdf5')

<h3>Make the Predictions</h3>

In [24]:
sumbit = best_model.predict(test)

<h3>Convert file in the right Structure</h3>

In [25]:
results = np.argmax(sumbit,axis = 1)
results = pd.Series(results,name="Label")

<h3>Save the predictions</h3>

In [26]:
submission = pd.concat([pd.Series(range(1,28001),name = "ImageId"),results],axis = 1)

submission.to_csv("submission.csv",index=False)